## Simple Searching

Wir lernen eine ziemlich universelle Methoden kennen, um Probleme zu lösen (nun ja, zumindest, es zu versuchen ;). Die grobe Idee ist einfach: Die Welt befindet sich zu jedem Betrachtungszeitpunkt (diskret, endlich viele!) in einem Zustand. Jeder Zustand läßt sich über endlich viele Elemente und Beziehungen zwischen diesen Elementen beschreiben. Sehr oft reichen endlich viele Werte, um die Zustände des Elemente und Beziehungen zu beschreiben (Reality check: immer!). Es gibt Operationen, die Operatoren instantiieren, um von einem Zustand zu einem Folgezustand überzugehen. Auch hier gehen wir von nur endlich vielen Operationen aus, die möglich bzw. plausibel sind. Nun können wir einen Startzustand angeben und eine Endzustand (oder eine Menge von Endzuständen). Die Lösung des Problems können wir nun durch Suche nach einem Weg vom Startzustand zu einem Endzustand finden.

*Beispiel*: **Kannibalen/Missionarsproblem**

Auf der westlichen Seite eines Flusses, der von Norden nach Süden fließt, befinden sich 3 Missionare, 3 Kannibalen und ein Boot für 1-2 Personen. Das Boot ist nicht selbstfahrend, ohne Fahrer treibt es ab und ist verloren!

Ihre Aufgabe ist es nun, die 3 Missionare und die 3 Kannibalen mit dem Boot vollständig auf die östliche Seite überzusetzen. Dabei müssen Sie die folgende Nebenbedingung beachten: sollten auf irgendeiner Seite des Flusses (die Besatzung im Boot zählt mit, sobald dieses in Ufernähe ist!) die Kannibalen einmal in der Überzahl sein, so werden die Missionare dort gefressen! Das ist dann kein gültiger Lösungsweg!

*Problemrepräsentierung*:

Wir beschreiben das Problem über Zustände $(m,k,b)$. Hierbei gibt $m$ die Anzahl der Missionare und $k$ die der Kannibalen auf der westlichen Seite des Flusses an, $b = 1$ sagt: das Boot ist auf der westlichen Seite, $b = 0$ sagt: das Boot ist auf der östlichen Seite des Flusses.

Der Startzustand ist $(3,3,1)$ (Alles ist westlich), der gewünschte Endzustand ist $(0,0,0)$ (Alles ist östlich).

Wir können das Boot fahren lassen, in dem wir Kannibalen und/oder Missionare ins Boot setzen. Wir gehen davon aus, dass die alle freiwillig zur anderen Seite fahren ;)

Zu einem Zustand $(m_1,k_1,b_1)$ ergeben sich mögliche Folgezustände durch die technisch möglichen Fahroperation. Erklären wir das per Beispiel (und zeigen es dann exakt als Programmcode):

$(3,3,1)$ hat die "denkbaren" Folgezustände $(2,2,0)$, $(2,3,0)$, $(3,2,0)$, $(1,3,0)$, $(3,1,0)$. Hier entsteht z.B. $(2,2,0)$ aus der Operationen "Lasse 1 Missionar und 1 Kannibalen mit dem Boot auf die andere Seite fahren" (die Richtung ergibt sich automatisch aus der Position des Bootes, deren Angabe können wir uns also sparen).

Manche der Folgezustände, die eintreten bzw. eintreten würden, sind nicht *valide*, oben sind es $(1,3,0)$ und $(2,3,0)$ (wir sollten also tunlichst nicht einen oder zwei Missionare allein auf die andere Seite schicken).

Generell darf westlich kein Zustand $(m,k,?)$ eintreten mit $0 < m < k$, und östlich, also für $(m,k,?)$, keiner mit $k < m < 3$ (die Lage des Bootes ist hier egal, ob das nun gerade im Westen oder Osten liegt, ist irrelvant dafür, ob der Zustand invalide ist, deshalb haben wir ein ? angegeben).

Das Ganze müssen wir jetzt noch mit einer systematischen Suche kombinieren, dann können wir das Problem lösen! Machen wir das:

In [43]:
startzustand = [3,3,1]
zielzustand  = [0,0,0]

# Erzeuge alle denkbaren Folgezustände, wir schauen nicht auf feasibility
# Wir geben eine Liste von Listen (den Zuständen) zurück
def gib_folgezustaende(zustand):
    result = []
    m,k,b = zustand
    b_new = 0 if b else 1    
    # now try all potential operations
    # Let's keep it simple for now:
    if b:
        if k > 0:
            if m > 0:
                result.append([m-1,k-1,b_new])
            if k > 1:
                result.append([m,k-2,b_new])
            result.append([m,k-1,b_new])       
        if m > 1:
            result.append([m-2,k,b_new])
        if m > 0:
            result.append([m-1,k,b_new]) 
    if not b:
        if k < 3:
            if m < 3:
                result.append([m+1,k+1,b_new])
            if k < 2:
                result.append([m,k+2,b_new])
            result.append([m,k+1,b_new])       
        if m < 2:
            result.append([m+2,k,b_new])
        if m < 3:
            result.append([m+1,k,b_new])
            
    return result

## Some tests
print(gib_folgezustaende([3,3,1]))            
print(gib_folgezustaende([2,2,0]))
print(gib_folgezustaende([2,2,1]))
print(gib_folgezustaende([1,1,0]))
print(gib_folgezustaende([1,1,1]))

[[2, 2, 0], [3, 1, 0], [3, 2, 0], [1, 3, 0], [2, 3, 0]]
[[3, 3, 1], [2, 3, 1], [3, 2, 1]]
[[1, 1, 0], [2, 0, 0], [2, 1, 0], [0, 2, 0], [1, 2, 0]]
[[2, 2, 1], [1, 3, 1], [1, 2, 1], [3, 1, 1], [2, 1, 1]]
[[0, 0, 0], [1, 0, 0], [0, 1, 0]]


In [66]:
## Jetzt checken wir noch, ob die Zustände valide sind!
def is_valid(zustand):
    m,k,b = zustand
    # es gibt im Westen mehr Kannibalen, als Missionare
    if m < k and m > 0: return False 
    # es gibt im Osten mehr Kannibalen, als Missionare
    if m > k and m < 3: return False
    return True

def gib_valide_folgezustaende(zustand):
    return [z for z in gib_folgezustaende(zustand) if is_valid(z)]

# Some tests
print(gib_valide_folgezustaende([3,3,1]))            
print(gib_valide_folgezustaende([2,2,0]))
print(gib_valide_folgezustaende([2,2,1]))
print(gib_valide_folgezustaende([1,1,0]))
print(gib_valide_folgezustaende([1,1,1]))

[[2, 2, 0], [3, 1, 0], [3, 2, 0]]
[[3, 3, 1], [3, 2, 1]]
[[1, 1, 0], [0, 2, 0]]
[[2, 2, 1], [3, 1, 1]]
[[0, 0, 0], [0, 1, 0]]


Erklären wir das einmal ein wenig:

Die "denkbaren" Folgezustände zu $[1,1,0]$ können wir wie folgt finden: das Boot liegt im Osten, wir haben im Westen 1 Missionar und 1 Kannibalen, im Osten 2 Missionare und 2 Kannibalen. Wir können also 1M1K, 2M, 2K, 1K und 1M von Osten nach Westen fahren lassen, insgesamt also:

$[[2, 2, 1], [3, 1, 1], [2, 1, 1], [1, 3, 1], [1, 2, 1]]$

Lassen wir nur einen Missionar fahren, dann sind im Osten zuviele Kannibalen. 

Lassen wir einen oder zwei Kannibalen fahren, dann sind im Westen zuviele Kannibalen.

Valide Folgezustände zu $[1,1,0]$ sind also $[[2, 2, 1], [3, 1, 1]]$.

Verwenden wir das nun, um das Problem mit Suche zu lösen. Wir versuchen hier zunächst in die Tiefe zu laufen. Was wir noch tun sollten,  ist, Zustände zu vermeiden, die wir schon einmal auf dem bisher zurückgelegten Weg besucht hatten (denken Sie dran: wir könnten beliebig oft hin und her fahren, wenn wir ein valide Operation gefunden haben...).

In [98]:
# Rekursive Suche in die Tiefe (depth-first search with chronolocigal backtracking)
def suche(zustand,history,all_solutions=False,level=0,debug=1):
    if debug: print(level*' ',zustand," ->",end="")
        
    # if compare(zustand,zielzustand): return (True,history+[zustand])
    if zustand == zielzustand: return (True,history+[zustand])
    fzustaende = gib_valide_folgezustaende(zustand)
    
    if debug: print("  ",fzustaende)
        
    if not len(fzustaende): return (False,[])
    for z in fzustaende:
        if z not in history+zustand:
            res1,res2 = suche(z,history+[zustand],all_solutions,level+1,debug)
            if res1: 
                if all_solutions:
                    print("Solution found: ",res1,res2)
                else:
                    return (res1,res2) # Just stop
        else:
            if debug == 2: print((level+1)*' '+"repeated",z)
    return (False,[])

suche(startzustand,[],debug=2) # One solution

 [3, 3, 1]  ->   [[2, 2, 0]]
  [2, 2, 0]  ->   [[3, 3, 1], [4, 2, 1]]
  repeated [3, 3, 1]
   [4, 2, 1]  ->   [[4, 1, 0], [2, 2, 0], [4, 0, 0]]
    [4, 1, 0]  ->   [[4, 2, 1], [4, 3, 1]]
    repeated [4, 2, 1]
     [4, 3, 1]  ->   [[3, 3, 0], [4, 2, 0], [4, 1, 0]]
      [3, 3, 0]  ->   [[4, 4, 1], [4, 3, 1]]
       [4, 4, 1]  ->   [[3, 3, 0], [4, 3, 0], [4, 2, 0]]
       repeated [3, 3, 0]
        [4, 3, 0]  ->   [[4, 4, 1]]
        repeated [4, 4, 1]
        [4, 2, 0]  ->   [[4, 3, 1], [4, 4, 1]]
        repeated [4, 3, 1]
        repeated [4, 4, 1]
      repeated [4, 3, 1]
      [4, 2, 0]  ->   [[4, 3, 1], [4, 4, 1]]
      repeated [4, 3, 1]
       [4, 4, 1]  ->   [[3, 3, 0], [4, 3, 0], [4, 2, 0]]
        [3, 3, 0]  ->   [[4, 4, 1], [4, 3, 1]]
        repeated [4, 4, 1]
        repeated [4, 3, 1]
        [4, 3, 0]  ->   [[4, 4, 1]]
        repeated [4, 4, 1]
       repeated [4, 2, 0]
     repeated [4, 1, 0]
   repeated [2, 2, 0]
    [4, 0, 0]  ->   [[4, 1, 1], [4, 2, 1]]
     [4, 1, 

(False, [])

In [82]:
# All solutions, debugging disabled
suche(startzustand,[],all_solutions=True,debug=0)

Solution found:  True [[3, 3, 1], [2, 2, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [0, 2, 1], [0, 0, 0]]
Solution found:  True [[3, 3, 1], [2, 2, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [1, 1, 1], [0, 0, 0]]
Solution found:  True [[3, 3, 1], [3, 1, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [0, 2, 1], [0, 0, 0]]
Solution found:  True [[3, 3, 1], [3, 1, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [1, 1, 1], [0, 0, 0]]


(False, [])

In [92]:
## Noch nicht besonders schön ist die Funktion für die Folgezustände
## Gestalten wir die ein wenig knapper und sehen gleich einen
### Maximalwert für die Anzahl vor
### Um es später leichter verwenden zu können, geben wir den Wert
### global vor
max_value = 3

def gib_folgezustaende(zustand):
    global max_value
    m,k,b = zustand
    incr  = -1 if b else +1
    b_new = 0 if b else 1
    
    fzustaende = [
        [m+incr,k+incr,b_new],
        [m+incr,k,b_new],
        [m,k+incr,b_new],
        [m+incr*2,k,b_new],
        [m,k+incr*2,b_new]
    ]
    # Entferne alle, die für m bzw. k kleiner 0 oder größer 3 sind
    return [[k,m,b] for k,m,b in fzustaende 
                if k >= 0 and k <= max_value and m >= 0 and m <= max_value]

## Some tests
print(gib_folgezustaende([3,3,1]))            
print(gib_folgezustaende([2,2,0]))
print(gib_folgezustaende([2,2,1]))
print(gib_folgezustaende([1,1,0]))
print(gib_folgezustaende([1,1,1]))

[[2, 2, 0], [2, 3, 0], [3, 2, 0], [1, 3, 0], [3, 1, 0]]
[[3, 3, 1], [3, 2, 1], [2, 3, 1]]
[[1, 1, 0], [1, 2, 0], [2, 1, 0], [0, 2, 0], [2, 0, 0]]
[[2, 2, 1], [2, 1, 1], [1, 2, 1], [3, 1, 1], [1, 3, 1]]
[[0, 0, 0], [0, 1, 0], [1, 0, 0]]


In [89]:
# All solutions, debugging disabled
suche(startzustand,[],all_solutions=True,debug=0)

Solution found:  True [[3, 3, 1], [2, 2, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [1, 1, 1], [0, 0, 0]]
Solution found:  True [[3, 3, 1], [2, 2, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [0, 2, 1], [0, 0, 0]]
Solution found:  True [[3, 3, 1], [3, 1, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [1, 1, 1], [0, 0, 0]]
Solution found:  True [[3, 3, 1], [3, 1, 0], [3, 2, 1], [3, 0, 0], [3, 1, 1], [1, 1, 0], [2, 2, 1], [0, 2, 0], [0, 3, 1], [0, 1, 0], [0, 2, 1], [0, 0, 0]]


(False, [])

Jetzt wollen wir prüfen, ob wir auch für 4 Missionare und 4 Kannibalen einen Weg hinüber finden!

In [93]:
# Wir brauchen eine allgemeinere Version von is_valid:
def is_valid(zustand):
    global max_value
    m,k,b = zustand
    # es gibt im Westen mehr Kannibalen, als Missionare
    if m < k and m > 0: return False 
    # es gibt im Osten mehr Kannibalen, als Missionare
    if m > k and m < max_value: return False
    return True

In [96]:
# Jetzt können wir schon suchen!
max_value = 4
suche([4,4,1],[],all_solutions=True,debug=2)

 [4, 4, 1]  ->   [[3, 3, 0], [4, 3, 0], [4, 2, 0]]
  [3, 3, 0]  ->   [[4, 4, 1], [4, 3, 1]]
  repeated [4, 4, 1]
   [4, 3, 1]  ->   [[3, 3, 0], [4, 2, 0], [4, 1, 0]]
   repeated [3, 3, 0]
    [4, 2, 0]  ->   [[4, 3, 1], [4, 4, 1]]
    repeated [4, 3, 1]
    repeated [4, 4, 1]
    [4, 1, 0]  ->   [[4, 2, 1], [4, 3, 1]]
     [4, 2, 1]  ->   [[4, 1, 0], [2, 2, 0], [4, 0, 0]]
     repeated [4, 1, 0]
      [2, 2, 0]  ->   [[3, 3, 1], [4, 2, 1]]
       [3, 3, 1]  ->   [[2, 2, 0]]
       repeated [2, 2, 0]
      repeated [4, 2, 1]
      [4, 0, 0]  ->   [[4, 1, 1], [4, 2, 1]]
       [4, 1, 1]  ->   [[4, 0, 0]]
       repeated [4, 0, 0]
      repeated [4, 2, 1]
    repeated [4, 3, 1]
  [4, 3, 0]  ->   [[4, 4, 1]]
  repeated [4, 4, 1]
  [4, 2, 0]  ->   [[4, 3, 1], [4, 4, 1]]
   [4, 3, 1]  ->   [[3, 3, 0], [4, 2, 0], [4, 1, 0]]
    [3, 3, 0]  ->   [[4, 4, 1], [4, 3, 1]]
    repeated [4, 4, 1]
    repeated [4, 3, 1]
   repeated [4, 2, 0]
    [4, 1, 0]  ->   [[4, 2, 1], [4, 3, 1]]
     [4, 2, 1]  -

(False, [])

Das geht nicht! Das ist auch nicht besonders überraschend, weil eine wichtige Operation in der Lösung ist, dass wir, nachdem wir alle Kannibalen auf die andere Seite gebracht haben (was wir tun müssen!), einen zurücksenden, und dann fahren 2 Missionare nach Osten, einer kommt mit einem Kannibalen zurück und dann fahren alle Missionare hinüber und der letzte Kannibale holt dann nach und nach seine Kumpels aus dem Westen. Ein analoges Vorgehen mit 4 Kannibalen können wir nicht zum Erfolg führen, s. oben!

Das gilt so dann für jede Erhöhung der Anzahlen! Eine Variation würde sich ergeben, wenn wir das Boot auf 3 Plätze erhöhen würden - können wir dann das Problem mit 4 Kannibalen und 4 Missionaren lösen?

**Aufgabe**: Testen Sie mit einer Variation des obigen Programms, ob es möglich ist, das 4-Kannibalen, 4 Missionare, 3-Platz-Boot-Problem zu lösen!

Jupyter-Notebook können Sie über https://www.anaconda.com/download/ installieren!
Sie können auch bei https://www.kaggle.com einen Account anlegen und dort "live" Notebooks ausprobieren (mit gewissen Beschränkungen).

In [97]:
## Hier ist Platz für ihre Lösung, senden Sie mir ein korrekte ausgeführtes Jupyter-Notebook, 
# entweder per Link zu github oder als gerendertes PDF!